In [1]:
import sys
sys.path.append('/home/mz1482/project/vt-bayesian-opt-bopt_debug/')
sys.path.append('/home/mz1482/project/vt-bayesian-opt-bopt_debug/my_bayes_opt/')
import bo_new
import pandas as pd
import numpy as np
from data_analysis import get_heart_bounds, correlation_coef, graph_3d, get_index
from graph import *
from BayesOptLib.bayes_opt.bayesian_optimization import BayesianOptimization
from RandomSampler import RandomSampler
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics.pairwise import euclidean_distances
matplotlib.use('Qt5Agg')  # or can use 'TkAgg', whatever you have/prefer

In [2]:
def black_box(x, y, z):
    """
    Represents a black box function to maximize for CC of two ECGs given an XYZ coordinate
    :param x:
    :param y: coordinates of the prediction
    :param z:
    :return: CC of that point and the target
    """
    sample_ecg = ecgs[get_index(np.array([x, y, z]),labels)]
    return abs(correlation_coef(target_ecg, sample_ecg))

def black_box_12_lead(x, y, z):
    """
    Represents a black box function to maximize for CC of two ECGs given an XYZ coordinate
    :param x:
    :param y: coordinates of the prediction
    :param z:
    :return: CC of that point and the target
    """
    sample_ecg = ecgs[get_index(np.array([x, y, z]),labels)]
    sample_ecg = np.reshape(sample_ecg, [12, -1])
#     target_ecg = np.reshape(target_ecg, [12, -1])
    nums = 0
    for i in range(12):
#         ccs = np.corrcoef(sample_ecg[i], target_ecg[i])[0, 1]
        ccs = abs(correlation_coef(target_ecg.reshape(12,-1)[i], sample_ecg[i]))
        if ccs > .9:
            nums += 1

#     return True if nums == 12 else False 
    return nums

In [3]:
data_path = "/home/mz1482/project/vt-bayesian-opt-bopt_debug/data/simu_data_4000/"
# Reading in the ECGs and labels
# aucs = pd.read_csv(data_path+"Heart1/Heart1_AUCS.csv", header=None).to_numpy()
ecgs = pd.read_csv(data_path+"Heart1/Heart1_SimuData_4000.csv", header=None).to_numpy()
labels = pd.read_csv(data_path+"Heart1/Coord1_4000.csv", header=None).to_numpy() / 1000
uvc = pd.read_csv(data_path+"UVC/Coord1_UVC_4000.csv", header=None).to_numpy()
init = 10
steps = 20

In [4]:
ecgs.shape

(4000, 7212)

In [ ]:
bounds = get_heart_bounds(labels)
# Pick out a sample to use as a target
tidx = np.random.randint(0, labels.shape[0])
target, target_ecg = labels[tidx], ecgs[tidx]
# optimizer = optimize_point(labels,bounds)  
optimizer = bo_new.mybo(f=black_box_12_lead,pbounds=bounds, real_set=labels)
gp,X,rs,predicted = optimizer.gpfit_12_lead(init_points=init, n_iter=steps,  acq="ucb", kappa = 1,kappa_decay=.75,
                 kappa_decay_delay=0)
# print(optimizer.predicted)
# graph_cc_distribution(target_ecg,ecgs,labels)
# init_gp_plot(init,gp,labels,X)
# predicted_visited(init,target,target_ecg,labels,ecgs,X,predicted)

Initialization
-----------------------------------------------------------------
 Step |   Time |      Value |         x |         y |         z | 
    1 | 00m00s |    1.00000 |    3.5067 |  -76.8661 |   -8.6279 | 
    2 | 00m00s |    0.00000 |   31.0602 |  -77.0779 |   17.0360 | 
    3 | 00m00s |    0.00000 |   13.6659 |  -72.9178 |   19.3183 | 
    4 | 00m00s |    1.00000 |   28.3457 |  -91.3452 |  -20.7242 | 
    5 | 00m00s |    0.00000 |   -0.5753 |   -9.3055 |    2.6197 | 
    6 | 00m00s |    0.00000 |  -30.3748 |  -43.6686 |  -15.2477 | 
    7 | 00m00s |    0.00000 |    5.5773 | -100.0120 |  -37.4261 | 
    8 | 00m00s |    0.00000 |  -31.5163 |  -83.1665 |   36.8677 | 
    9 | 00m00s |    0.00000 |    2.3004 |  -71.7836 |   29.7499 | 
   10 | 00m00s |    0.00000 |  -33.2249 | -102.0960 |   11.3615 | 
Bayesian Optimization
-----------------------------------------------------------------
 Step |   Time |      Value |         x |         y |         z | 
1
   11 | 00m11s |    7.000

In [ ]:
target